# Carga y manipulación de datos con pandas

Pandas es una de las bibliotecas de análisis de datos más útiles de Python. Ha sido fundamentales para aumentar el uso de Python en la comunidad de la ciencia de datos.

Es desarrollada por [Wes McKinney](https://github.com/wesm). 

Está construida con el paquete Numpy y su estructura de datos clave es llamada el **DataFrame**. 

El DataFrame permite almacenar y manipular datos tabulados en filas de observaciones y columnas de variables.

El DataFrame es una estructura que permite datos heterogéneos. 

In [ ]:
# Importar pandas
import pandas as pd
import matplotlib.pyplot as plt

### Un dataframe sencillo

In [ ]:
df = pd.DataFrame([[1, 2.4], [3, 4.9]], index=['A', 'B'], columns=['X', 'Y'])
df

## Cargando los datos y explorándolos

Se trabajará sobre un archivo de datos metereológicos de la Consejeria Agricultura Pesca y Desarrollo Rural Andalucía.

In [ ]:
from IPython.display import IFrame
IFrame('http://www.juntadeandalucia.es/agriculturaypesca/ifapa/ria/servlet/FrontController?action=Static&url=coordenadas.jsp&c_provincia=4&c_estacion=4', width=700, height=350)

In [ ]:
# Ver el contenido del archivo
!type ..\data\tabernas_meteo_data.txt

In [ ]:
pd.read_csv("../data/tabernas_meteo_data.txt").head(5)

Al leer el documento se pueden identificar dos dificultades para importarlo a memoria: delimitadores y headerlines

Concretamente, se tienen que hacer lo siguiente:

* Manejar el número variable de espacios en blanco entre columnas.
* Saltar las primeras líneas.
* Dar nombres nuevos a las columnas.
* Descartar la columna del día del año
* Parsear las fechas en el formato correcto.

La forma clásica para resolver este problema es hacer una lectura línea a línea **parseando** los datos del archivo de acuerdo al formato que se espera recibir.

`pandas` a través de la función `read_csv` permite manejar esta situación.

> [pandas.to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)

In [ ]:
df = pd.read_csv(
    "../data/tabernas_meteo_data.txt",
    delim_whitespace=True,   # delimitado por espacios en blanco
    usecols=(0, 2, 3, 4, 5),  # columnas que queremos usar
    skiprows=2,  # saltar las dos primeras líneas
    names=['DATE', 'TMAX', 'TMIN', 'TMED', 'PRECIP'],
    parse_dates=['DATE'],
    dayfirst=True,  # ¡Importante
    index_col=["DATE"]  # indexar por fechas
)

# Ordenar de más antigua a más reciente
df.sort_index(inplace=True)

# Mostrar las primeras o las últimas líneas
df.head()

In [ ]:
# Comprobar los tipos de datos de la columnas
df.dtypes

In [ ]:
# Obtener información general del dataset
df.info()

En una dataframe pueden existie datos de tipo diferente en las columnas: en este ejemplo, fechas (como índice) y (flotantes en las columnas). El que un dato sea de tipo fecha y no un string u otro tipo, permite obtener información como el día de la semana de manera directa:

In [ ]:
df.index.dayofweek

 ### Descripción estadística
Para calcular algunas estadísticas básicas de los datos en cada columna se usa la siguiente instrucción:

In [ ]:
df.describe()

## Accediendo a los datos 

### Columnas

Hay dos formas de acceder a las columnas: **por nombre** o **por atributo** (si no contienen espacios ni caracteres especiales).

In [ ]:
# Accediendo con clave
df['TMAX'].head()

In [ ]:
# Accediendo con atributo
df.TMIN.head()

In [ ]:
# Accediendo a varias columnas a la vez
df[['TMAX', 'TMIN']].head()

#### Modificando valores de columnas

In [ ]:
(df[['TMAX', 'TMIN']] / 10).head()

#### Aplicar una función a una columna (ej. np.mean)

In [ ]:
import numpy as np
np.mean(df.TMAX)

In [ ]:
# Calcular la media con pandas
df.TMAX.mean()

### Filas 

Para acceder a las filas existen dos métodos: `.loc` (usando etiquetas), `.iloc` (usando enteros) ~~y `.ix` (que combina ambos)~~ (`.ix` ha desaparecido en la versión 0.20).

In [ ]:
# Acceder a una fila por índice
df.iloc[1]

In [ ]:
# Accediendo a una fila por etiqueta
df.loc["2016-09-02"]

Puedo incluso hacer secciones basadas en fechas:

#### Crear una sección con una fecha

In [ ]:
df.loc["2016-12-01":]

### Filtrando los datos 

También se puede indexar utilizando arreglos de valores booleanos, por ejemplo procedentes de la comprobación de una condición:

In [ ]:
# Comprobando que registros carecen de datos válidos
df.TMIN.isnull().head()

In [ ]:
# Accediendo a los registros que cumplen una condición
df.loc[df.TMIN.isnull()]

In [ ]:
# Valores de precipitación por encima de la media:
print(df.PRECIP.mean())
df[df.PRECIP > df.PRECIP.mean()]

### Creación de nuevas columnas 

In [ ]:
# Agrupar por año y día: crear dos columnas nuevas
df['year'] = df.index.year
df['month'] = df.index.month

## Creando agrupaciones 

En muchas ocasiones se requiere realizar agrupaciones de datos en base a determinados valores como son fechas, o etiquetas (por ejemplo, datos que pertenecen a un mismo ensayo o lugar)

Para la agrupación de datos se utiliza el método:`groupby`:

In [ ]:
# Crear la agrupación
monthly = df.groupby(by=['year', 'month'])

In [ ]:
# ver los grupos que se han creado
monthly.groups.keys()

Con estos grupos se puede realizar lo siguiente:

* Acceder a los datos individualmente (por ejemplo, comprobar qué pasó cada día de marzo de 2016) 
* Realizar una reducción de datos, para comparar diversos grupos (por, ejemplo caracterizar el tiempo de cada mes a lo largo de los años)

In [ ]:
# Acceder a un grupo
monthly.get_group((2016,3)).head()

In [ ]:
# Realizar una agregación de los datos:
monthly_mean = monthly.mean()
monthly_mean.head(24)

## Graficación

## Líneas

In [ ]:
# Graficar las temperaturas máx, min, med
df.plot(y=["TMAX", "TMIN", "TMED"])
plt.title('Temperaturas');

## Cajas

In [ ]:
df.loc[:, 'TMAX':'PRECIP'].plot.box();

## Graficando los datos de un "típíco día `d` del mes `m` del año `a`

Gtaficar la temperatura máxima de las máximas, mínima de las mínimas, media de las medias para cada día del año de los años disponnibles

In [ ]:
group_daily = df.groupby(['month', df.index.day])

daily_agg = group_daily.agg({'TMED': 'mean', 'TMAX': 'max', 'TMIN': 'min', 'PRECIP': 'mean'})
daily_agg.head(50)

In [ ]:
daily_agg.plot(y=['TMED', 'TMAX', 'TMIN'])

### Matriz de dispersion

In [ ]:
## Visualizaciones especiales

# scatter_matrix
from pandas.plotting import scatter_matrix
axes = scatter_matrix(df.loc[:, "TMAX":"TMED"])